Conversion of transformer model to ct2:


ct2-transformers-converter --model /mnt/c/Projects/models/whisper/whisper-full-ft-no-PL/checkpoint-2353 --output_dir /mnt/c/Projects/models/whisper/whisper-full-ft-no-PL-ct2 --quantization float16

https://huggingface.co/docs/peft/main/en/package_reference/lora#peft.LoraModel.merge_and_unload

In [1]:
model_dir = "/mnt/c/Projects/models/whisper/whisper-full-ft-no-PL-ct2"

In [2]:
from faster_whisper import WhisperModel, BatchedInferencePipeline

model = WhisperModel(model_dir, device="cuda", compute_type="float16")
batched_model = BatchedInferencePipeline(model=model)

/mnt/c/Projects/code/faster-whisper-inference/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os

data = "/mnt/c/Projects/datasets/PORT/set_1_2_test_manifest.json"
base_dir = os.path.dirname(data)

In [4]:
import json
from typing import List, Dict

def load_manifest_nemo(input_manifest_path: str) -> List[Dict[str, str]]:
    """
    loads the manifest file in Nvidia NeMo format to process the entries and store them
    into a list of dictionaries

    input_manifest_path: the manifest path that contains the information of the audio
    clips of interest
    ---
    returns: a list of dictionaries of the information in the input manifest file
    """

    dict_list = []

    with open(input_manifest_path, "r+", encoding="utf-8") as f:
        for line in f:
            dict_list.append(json.loads(line))

    return dict_list

In [5]:
df = load_manifest_nemo(data)

In [6]:
base_dir

'/mnt/c/Projects/datasets/PORT'

In [7]:
for item in df:
    item['audio_filepath'] = os.path.join(base_dir, item['audio_filepath'])


In [8]:
df[:2]

[{'audio_filepath': '/mnt/c/Projects/datasets/PORT/mms_set_1/test_split/mms/test/CHDIR_154_2022-04-04_18-00000000-00007597.wav',
  'root_file': 'CHDIR_154_2022-04-04_18.wav',
  'duration': 7.597,
  'start': 0.0,
  'end': 7.597,
  'text': 'TUGBOAT SOL TEN ELEVEN TUGBOAT SOL TEN ELEVEN MOTOR TANKER ACHILLEAS IS CALLING ON SEVEN THREE'},
 {'audio_filepath': '/mnt/c/Projects/datasets/PORT/mms_set_1/test_split/mms/test/CHDIR_154_2022-04-04_18-00007597-00012685.wav',
  'root_file': 'CHDIR_154_2022-04-04_18.wav',
  'duration': 5.088,
  'start': 7.597,
  'end': 12.685,
  'text': 'SOL ONE ZERO ONE ONE V T I S WEST SOL ONE ZERO ONE ONE'}]

In [9]:
audio_filepaths = []

for item in df:
    audio_filepaths.append(item['audio_filepath'])

In [10]:
import av
av.open('/mnt/c/Projects/datasets/PORT/mms_set_1/test_split/mms/test/CHDIR_154_2022-04-04_18-00000000-00007597.wav') 

<av.InputContainer '/mnt/c/Projects/datasets/PORT/mms_set_1/test_split/mms/test/CHDIR_154_2022-04-04_18-00000000-00007597.wav'>

In [11]:
audio_filepaths[1]

'/mnt/c/Projects/datasets/PORT/mms_set_1/test_split/mms/test/CHDIR_154_2022-04-04_18-00007597-00012685.wav'

In [12]:
segments, info = model.transcribe(audio_filepaths[1], beam_size=5)


: 

In [26]:
segments, info = batched_model.transcribe(audio = audio_filepaths[:1], batch_size=2)


ValueError: File object has no read() method, or readable() returned False.